In [192]:
# imPORT LIBRARIES 
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from math import pi, sqrt
import handcalcs.render
import pandas as pd
import json 
# !pip install handcalcs

In [193]:
# CONFIGURATIONS
#help(handcalcs.set_option)
handcalcs.set_option("latex_block_start", "$")
handcalcs.set_option("latex_block_end", "$")
handcalcs.set_option("math_environment_end", "aligned")
handcalcs.set_option("use_scientific_notation",True)
#handcalcs.set_option("line_break", "\\\\[20pt]")
#handcalcs.set_option("param_columns", 1)

#handcalcs.set_option("math_environment_start", "aligned")
#handcalcs.set_option("math_environment_end", "aligned")
#handcalcs.set_option("line_break", r"<br>")
#handcalcs.set_option("line_break", r"\\")

In [194]:
from localcode import *
fig_counter=1

# Resonance Tank Capacitor and Output Capacitor

## Inputs and claculated parameters

In [195]:
with open("../01_llc_tank/04_outputs/out_results.json", "r", encoding="utf-8") as f:
    out_datas = json.load(f)

# Load variables 
for key, value in out_datas.items():
    globals()[key] = value
    
params = ['n',  'Lm_uH', 'fsw_min', 'fsw_max', 'Ir_rms', 'Im_rms', 'Ioe_rms', 'Ios_rms', 'L_second_uH']

dic_transfo= {x:out_datas[x] for x in params}
Al = out_datas["Lm_uH"]/out_datas["n"]**2
dic_transfo["Al_uH"]= Al

S= pd.Series(dic_transfo, name = "Transormer parameters").to_frame()
display(S)

,Transormer parameters
n,4.000
Lm_uH,65.392
fsw_min,60170.000
fsw_max,156220.000
Ir_rms,10.354
Im_rms,6.992
Ioe_rms,7.636
Ios_rms,30.545
L_second_uH,4.087
Al_uH,4.087


In [196]:
with open("../01_llc_tank/04_outputs/inp_datas.json", "r", encoding="utf-8") as f:
    inp_datas = json.load(f)



# Load variables 
for key, value in inp_datas.items():
    globals()[key] = value


## Output capacitor Co

### Formulas

For a resistive load, the output capacitor is determined by
$$
C_o \geq C_{o_{min}} = \frac{I_o}{8 \cdot f_{sw} \cdot \Delta V_{out}}
$$

Where:

- $C_o$ : Output capacitance (Farads, F)  
- $I_o$ : Output load current (Amperes, A)  
- $f_{sw}$ : Switching frequency of the converter (Hertz, Hz)  
- $\Delta V_{out}$ : Maximum allowed output voltage ripple (Volts, V)  
$$ESR_{max} = \frac{\Delta V_{out}}{I_{rect\_peak}} = \frac{\Delta V_{out}}{\frac{\pi}{4} \cdot I_o\cdot{2}}  $$
$$I_{C_o} = I_o \sqrt{\frac{\pi^2}{8} - 1}$$

Where:

- $I_{C_o}$ : RMS current of the capacitor @ f_sw = f_nom  
 
See page 26, 27 [\[1\]](#referencesID_03_1).
<br> By default, you can found all formula of this chapeter in the same ref 

### Numerical Implementation of the Formulas

**We must use fsw_min to the worst case**

In [197]:
%%render
Io # A
fsw_min # Hz
DV = 0.25 # vpp
Co_min_uF = 1e6*Io/(8*fsw_min*DV ) # uF
ESR_max_m = 1e3*DV /(Io*pi/2) # mΩ 
I_Co= Io*sqrt(pi**2/8-1) # Arms

<IPython.core.display.Latex object>

### First proposition

B40910A8127M000 aluminum electrolytic capacitors with Temp_max = 150°C, Below a screenshoot of the datasheet [\[2\]](#referencesID_03_2).

In [198]:
first_part_path = "./03_figures/"

fig_counter = show_figure_pair_with_captions(
    fig_counter, 
    "B40910.png",
    "VRMS",
    first_part_path= first_part_path, 
)

Let's start with a configuration of 5 capacitors in parallel

In [199]:
%%render 
#4 parallels capascitor
Nb_capa= 5
C_capa_nom = 120 # uF
Margin = 20 #\%
C_capa = C_capa_nom*(1-Margin/100)# uF, Worst case 
C_eq = C_capa*Nb_capa # uF > 208uF ok
ESR_capa= 17 # Ω @ 100kHz

ESR_eq = ESR_capa/Nb_capa # Ω < 6.36m OK
Icapa_max = 4.6
Ieq_max = Icapa_max *Nb_capa  #  @ Arms @ 125°C 100kHz > 12.08Arms ok

<IPython.core.display.Latex object>

**Voltage ripples**

In [200]:
%%render 
Delta_V_out = Io/(8*fsw_min*C_eq*1e-6 )

<IPython.core.display.Latex object>

#### Self heating

**Power Dissipation of Each Capacitor**

In [201]:
%%render 
I_each_capa= I_Co/Nb_capa
P_selfHeating = ESR_capa*1e-3*I_each_capa**2 # W

<IPython.core.display.Latex object>

**Estimation of Thermal Resistance Rth**

In [202]:
%%render 
# Estimation or Rth 

# @100Khz and 125°C Iac,R I=4.6A
# 100KHZ 20°C ESR = 0.017 Ohm 
# Max T = 150°C 
Delta_T= 150-125 # °C
ESR = 17e-3
I = 4.6 # Arms
P_dissip= ESR*I**2 # W
R_th = Delta_T/P_dissip # °C/W

<IPython.core.display.Latex object>

**The self heating estimation and the max ambiant temp**

In [203]:
%%render 
# self heating 
Delta_T= P_selfHeating*R_th # °C => low delta temp
Margin = 30 # °C 
T_max = 150
T_amb_max = T_max -Delta_T-Margin  # °C 

<IPython.core.display.Latex object>

**Voltage margin**

In [204]:
%%render  
Vo_max # VDC
V_max_datasheet = 63 # VDC
Voltage_Margin =  100*(V_max_datasheet-Vo_max) /V_max_datasheet # \%

<IPython.core.display.Latex object>

This solution is acceptable, but the voltage margin is limited.

In [205]:
dic_capa1= {}
for x in "C_capa,ESR_eq,C_eq,Ieq_max,Nb_capa,P_selfHeating,T_amb_max,Voltage_Margin,Delta_V_out".split(","):
    dic_capa1[x]= eval(x)

### Second proposition

EMHS101ARA331MMN0S aluminum electrolytic capacitors with Temp_max = 150°C, Below a screenshoot of the datasheet [\[4\]](#referencesID_03_4).

In [206]:
first_part_path = "./03_figures/"

fig_counter = show_figure_pair_with_captions(
    fig_counter, 
    "EMHS.png",
    "VRMS" , 
    first_part_path= first_part_path, 
)

Let's start with a configuration of 5 capacitors in parallel

In [207]:
%%render 
#4 parallels capascitor
Nb_capa= 10
C_capa_nom = 330 # uF
Margin = 20 #\%
C_capa = C_capa_nom*(1-Margin/100)# uF, Worst case 
C_eq = C_capa*Nb_capa # uF > 208uF ok
ESR_capa= 59 # Ω @ 100kHz

ESR_eq = ESR_capa/Nb_capa # Ω < 6.36m OK
Icapa_max = 2.3
Ieq_max = Icapa_max *Nb_capa  #  @ Arms @ 125°C 100kHz > 12.08Arms ok

<IPython.core.display.Latex object>

**Voltage ripples**

In [208]:
%%render 
Delta_V_out = Io/(8*fsw_min*C_eq*1e-6 )

<IPython.core.display.Latex object>

#### Self heating

**Power Dissipation of Each Capacitor**

In [209]:
%%render 
I_each_capa= I_Co/Nb_capa
P_selfHeating = ESR_capa*1e-3*I_each_capa**2 # W

<IPython.core.display.Latex object>

**Estimation of Thermal Resistance Rth**

In [210]:
%%render 
# Estimation or Rth 

# @100Khz and 125°C Iac,R I=4.6A
# 100KHZ 20°C ESR = 0.017 Ohm 
# Max T = 150°C 
Delta_T= 150-125 # °C
P_dissip= ESR_capa*1e-3*Icapa_max **2 # W
R_th = Delta_T/P_dissip # °C/W

<IPython.core.display.Latex object>

**The self heating estimation and the max ambiant temp**

In [211]:
%%render 
# self heating 
Delta_T= P_selfHeating*R_th # °C => low delta temp
Margin = 30 # °C 
T_max = 150
T_amb_max = T_max -Delta_T-Margin  # °C 

<IPython.core.display.Latex object>

In [212]:
%%render  
Vo_max # VDC
V_max_datasheet = 100 # VDC
Voltage_Margin =  100*(V_max_datasheet-Vo_max) /V_max_datasheet # \%

<IPython.core.display.Latex object>

In [213]:
dic_capa2= {}
for x in "C_capa,ESR_eq,C_eq,Ieq_max,Nb_capa,P_selfHeating,T_amb_max,Voltage_Margin,Delta_V_out".split(","):
    dic_capa2[x]= eval(x)

### Comparaison

In [214]:
dic_capa1["VmaxDC"]=63
dic_capa2["VmaxDC"]=100

In [215]:
requirements= {"ESR_eq":ESR_max_m, "C_eq":Co_min_uF, "Ieq_max":I_Co, 
               "VmaxDC":Vo_max, "Delta_V_out":DV}

In [216]:
pd.DataFrame([requirements, dic_capa1, dic_capa2], 
             index= ["requirements","Solutio1", "Solutio2"]).T.round(2)

,requirements,Solutio1,Solutio2
ESR_eq,6.37,3.40,5.90
C_eq,207.74,480.00,2640.00
Ieq_max,12.09,23.00,23.00
VmaxDC,54.00,63.00,100.00
Delta_V_out,0.25,0.11,0.02
C_capa,NaN,96.00,264.00
Nb_capa,NaN,5.00,10.00
P_selfHeating,NaN,0.10,0.09
T_amb_max,NaN,113.10,113.10
Voltage_Margin,NaN,14.29,46.00


The margin in the first solution is limited (16% in the worst case). However, I propose we proceed with this option, given the number of parallel capacitors. Care must be taken in control to prevent any overshoot or transient voltage, especially when Vout is at Vout_max.

## Transformer design

In [217]:
with open("../01_llc_tank/04_outputs/out_results.json", "r", encoding="utf-8") as f:
    out_datas = json.load(f)


# Load variables 
for key, value in out_datas.items():
    globals()[key] = value
    
params = ['n',  'Lm_uH', 'fsw_min', 'fsw_max', 'Ir_rms', 'Im_rms', 'Ioe_rms', 'Ios_rms', 'L_second_uH']

dic_transfo= {x:out_datas[x] for x in params}
Al = out_datas["Lm_uH"]/out_datas["n"]**2
dic_transfo["Al_uH"]= Al

S= pd.Series(dic_transfo, name = "Transormer parameters").to_frame()
display(S)

,Transormer parameters
n,4.000
Lm_uH,65.392
fsw_min,60170.000
fsw_max,156220.000
Ir_rms,10.354
Im_rms,6.992
Ioe_rms,7.636
Ios_rms,30.545
L_second_uH,4.087
Al_uH,4.087


where: 
* `n`: Transformer turns ratio $ n = N_p / N_s $
* `N_p = n` since `N_s= 1`
* `Lm_uH`: Magnetizing inductance $ L_m $, sets magnetizing current and contributes to gain shaping
* `fsw_min`: Minimum switching frequency $ f_{sw,\min} $, corresponds to maximum load
* `fsw_max`: Maximum switching frequency $ f_{sw,\max} $, corresponds to no-load
* `Ir_rms`: RMS current in the resonant tank inductor $ I_r = \sqrt{I_m^2 + I_{oe}^2} $
* `Im_rms`: RMS magnetizing current $ I_m $, flows through $ L_m $
* `Ioe_rms`: RMS transferred current in the primary winding, represents reflected load current
* `Ios_rms`: RMS current in the secondary winding $ I_{os} = I_{oe} \cdot n $
* `Al_uH`: Inductance per turn squared $ A_L $, used to compute turns: $ L = A_L \cdot N^2 $


**Bpeak formula**

$$
B_{peak} = \frac{L_m \cdot Im_{peak}}{N_p \cdot A_e}= \frac{L_m \cdot \sqrt{2} \cdot  Im_{rms}}{N_p \cdot A_{emin}}
$$

Where
- `A_e` = effective core area (in m²)
- `A_emin` = effective minimum core area (in m²)
- `N_p` = primary turnes


The `Al` value: 

In [218]:
%%render long
Al_nH= 1e3*Lm_uH/n**2 # nH/turn squared

<IPython.core.display.Latex object>

**ELP 43/10/28 core**

In [219]:
%%render long
Lm_uH
Np= n # turns 
Ae = 225 # mm2 see datasheet ELP 43/10/28
Aemin = 217 # mm2 see datasheet ELP 43/10/28
Im_rms # Arms
B_peak = 1e3*Lm*sqrt(2)*Im_rms/(Np*Aemin*1e-6) # mT > Bs= 250mT NOK

<IPython.core.display.Latex object>

Since the formulat of peak flux density is $
\begin{aligned}
B_{peak} = \mathrm{Lm} \cdot \sqrt { 2 } \cdot \frac{ \mathrm{Im}_{rms} }{ \mathrm{Np} \cdot \mathrm{Aemin} }   
\end{aligned}
$
We can reduct the peak by reducing the current or by reducing Lm or by increasing Np 

We can reduce $A_L$ by adding an air gap, below the formula of air gap length

$$
A_{L}' = \frac{A_L}{1 + \mu_r \,\frac{L_g}{L_e}}
$$

$$
l_g = \frac{L_e}{\mu_r} \left( \frac{A_L}{A_L'} - 1 \right)
$$

Where
- $ A_{L}' $ : the corrected inductance factor (in nH or µH), after introducing the air gap
- $ A_L $ : the initial inductance factor provided by the core manufacturer (in nH or µH)
- $ l_g $ : the air gap length (in mm)
- $ L_e $ : the effective magnetic path length of the core (in mm)

This equation accounts for the effect of the air gap on the inductance. Increasing the air gap reduces the effective permeability, and thus reduces $ A_{L}' $.


In [220]:
dic={"core":"ELP 43/10/28 with I 43/4/28 N87",
 "Al_nH":7300, 
"Aemin":217,
"Le":61.6 ,
"µe":1560 ,
"K1":358,
"K2":-0.794,    
"K3_25":621, 
"K4_25":-0.796,
"Bsat": 300}
core ="ELP 43/10/28__AND__I 43/4/28 N87"
Aemin = dic['Aemin']
K1 = dic["K1"]
K2 = dic["K2"]
K3_25 = dic["K3_25"]
K4_25 = dic["K4_25"]
Al_nH0 = dic["Al_nH"]
Le = dic["Le"]
mu_e = dic["µe"]


In [221]:
%%render long
core
Aemin 
K1 
K2 
K3_25 
K4_25 
Al_nH0 
Le 
mu_e 
s_mm= (Al_nH/K1)**(1/K2) # mm
lg_mm= Le *(Al_nH0/Al_nH-1)/mu_e # mm
I_dc = (0.9*Al_nH/K3_25)**(1/K4_25)
B_peak = 1e3*Lm*sqrt(2)*Im_rms/(Np*Aemin*1e-6)

<IPython.core.display.Latex object>

So, the estimated air gap $s_{\mathrm{mm}}$ using the datasheet constants $K_1$ and $K_2$ is slightly greater than the calculated $l_{g\_\mathrm{mm}}$ air gap obtained from the simple formula.

In [222]:
# https://www.tdk-electronics.tdk.com/inf/80/db/fer/elp_22_6_16.pdf
# https://www.tdk-electronics.tdk.com/inf/80/db/fer/elp_32_6_20.pdf
# https://www.tdk-electronics.tdk.com/inf/80/db/fer/elp_38_8_25.pdf
# https://www.tdk-electronics.tdk.com/inf/80/db/fer/elp_43_10_28.pdf
# https://www.tdk-electronics.tdk.com/inf/80/db/fer/elp_58_11_38.pdf
# https://www.tdk-electronics.tdk.com/inf/80/db/fer/elp_64_10_50.pdf
# https://www.tdk-electronics.tdk.com/inf/80/db/fer/e_70_33_32.pdf
# https://www.tdk-electronics.tdk.com/inf/80/db/fer/e_80_38_40.pdf

In [223]:
dfcore = pd.DataFrame([
    
{"core":"ELP 43/10/28 with I 43/4/28 N87",
 "Al_nH":7300, 
"Aemin":217,
"Le":61.6 ,
"µe":1560 ,
"K1":358,
"K2":-0.794,    
#"K3_25":621, 
#"K4_25":-0.796,
"Bsat": 300}
, 

{"core":"ELP 58/11/38 with I 58/4/38 N87",
 "Al_nH":7400, 
"Aemin":308,
"Le": 80.7 ,
"µe":1540 ,
"K1":591,
"K2":-0.685, 
#"K3_25":791, 
#"K4_25":-0.796,
"Bsat": 300},



{"core":"ELP 64/10/50 with I 64/5/50 N87",
"Al_nH": 14000, 
"Aemin":518,
"Le": 69.7,
"µe": 1450, 
"K1":835,
"K2":-0.790, 
#"K3_25":1316, 
#"K4_25":-0.796, 
"Bsat":300},


    
{"core":"ELP 64/10/50 with ELP 64/10/50 N87",
 "Al_nH":12500, 
"Aemin":518,
"Le": 79,
"µe": 1490, 
"K1":820,
"K2":-0.767, 
"Bsat":300},

    
{"core":"ELP 64/10/50 with ELP 64/10/50 N95",
 "Al_nH":15500, 
"Aemin":518,
"Le": 79,
"µe": 1880, 
"Bsat":320},

{"core":"ELP 102/20/38 with I 102/7/38 N87",
 "Al_nH":9300, 
"Aemin":524.5,
"Le": 121.2,
"µe":1680 ,  
"Bsat":300
}

])

for i in range (len(dfcore)):
    dic  =dfcore.loc[i,:].to_dict()
    Aemin = dic['Aemin']
    K1 = dic["K1"]
    K2 = dic["K2"]
    #K3_25 = dic["K3_25"]
    #K4_25 = dic["K4_25"]
    Al_nH0 = dic["Al_nH"]
    Le = dic["Le"]
    µe = dic["µe"]
    s_mm= (Al_nH/K1)**(1/K2) # airgap
    lg_mm= Le *(Al_nH0/Al_nH-1)/µe
    #I_dc = (0.9*Al_nH/K3_25)**(1/K4_25)
    B_peak = 1e3*Lm*sqrt(2)*Im_rms/(Np*Aemin*1e-6)
    
    µ0= 4*pi **1e-7
    N=4
    lg = lg_mm*1e-3
    le = Le*1e-3
    Bpeak2 = µ0*N*sqrt(2)*Im_rms/(lg+le/µe)
    
    dfcore.loc[i,"s_mm"]=s_mm
    dfcore.loc[i,"lg_mm"]=lg_mm
    dfcore.loc[i,"s_div_e"]=s_mm/lg_mm
  
    dfcore.loc[i,"B_peak"]=B_peak
    dfcore.loc[i,"Bpeak2"]=Bpeak2
    #dfcore.loc[i,"Idc_25"]=I_dc
    dfcore.loc[i,"Bpeak_inf_Bsat"]=dic["Bsat"]>B_peak

In [224]:
dfcore.set_index("core")

,Al_nH,Aemin,Le,µe,K1,K2,Bsat,s_mm,lg_mm,s_div_e,B_peak,Bpeak2,Bpeak_inf_Bsat
core,,,,,,,,,,,,,
ELP 43/10/28 with I 43/4/28 N87,7300,217.0,61.6,1560,358.0,-0.794,300,0.046570,0.031043,1.500191,744.940031,2.243170e+06,False
ELP 58/11/38 with I 58/4/38 N87,7400,308.0,80.7,1540,591.0,-0.685,300,0.059428,0.042479,1.399007,524.844113,1.667464e+06,False
ELP 64/10/50 with I 64/5/50 N87,14000,518.0,69.7,1450,835.0,-0.790,300,0.133949,0.116591,1.148878,312.069472,9.608338e+05,False
ELP 64/10/50 with ELP 64/10/50 N87,12500,518.0,79.0,1490,820.0,-0.767,300,0.123167,0.109141,1.128515,312.069472,9.756415e+05,False
ELP 64/10/50 with ELP 64/10/50 N95,15500,518.0,79.0,1880,NaN,NaN,320,NaN,0.117345,NaN,312.069472,9.927506e+05,True
ELP 102/20/38 with I 102/7/38 N87,9300,524.5,121.2,1680,NaN,NaN,300,NaN,0.092019,NaN,308.202072,9.637509e+05,False


In [225]:
dic_trans_core= dfcore[dfcore.core=="ELP 64/10/50 with I 64/5/50 N87"].iloc[0,:].to_dict()
keys = ['core', 'Al_nH', 'Aemin', 'Le', 'µe', 'Bsat', 's_mm' , 'B_peak']
dic_trans_core = {k:dic_trans_core[k] for k in keys}
dic_trans_core 

{'core': 'ELP 64/10/50 with I 64/5/50 N87',
 'Al_nH': 14000,
 'Aemin': 518.0,
 'Le': 69.7,
 'µe': 1450,
 'Bsat': 300,
 's_mm': 0.1339488641172734,
 'B_peak': 312.06947242700016}

In [226]:
dfcores = pd.read_csv("FerriteCores.csv",sep=";").iloc[:,:-3]
dfcores

,core,Aemin(mm2),Le(mm),Ve(mm3),Bsat mT,Loss (W/set),K1,K2,µe,Al(nH),price by set
0,E64/10/50-3C95,519.0,79.9,40700,330.0,20.0,NaN,NaN,NaN,17000,11.05
1,ELP 43/10/28 with I 43/4/28 N87,217.0,50.8,11430,350.0,7.8,390.0,-0.784,1480.0,850,2.61
2,ELP 64/10/50 with I 64/5/50 N87,512.0,69.7,36200,300.0,4.8,835.0,-0.790,1450.0,14000,NaN
3,ELP 64/10/50 with ELP 64/10/50 N95,518.0,79.9,41500,320.0,5.1,NaN,NaN,1880.0,15500,14.52
4,ELP 64/10/50 with ELP 64/10/50 N87,518.0,79.9,41500,300.0,5.5,820.0,-0.767,1490.0,12500,11.27
5,E64/10/50 + PLT64/50/5 3C95,519.0,69.7,35500,330.0,17.0,NaN,NaN,NaN,18500,NaN
6,ER64/13/51-3C92 3C95,507.0,93.0,52600,330.0,25.0,NaN,NaN,NaN,17100,19.62
7,E100/60/28 N87,690.0,274.0,201390,NaN,4.7,NaN,NaN,1930.0,6500,17.40
8,ELP 102/20/38 with ELP 102/20/38 N87,524.5,147.6,79410,NaN,11.0,NaN,NaN,1790.0,8200,12.88
9,ELP 102/20/38 with ELP 102/20/38 N97,524.5,147.6,79410,NaN,9.7,NaN,NaN,1855.0,8500,14.07


In [227]:
mask  = dfcores["Bsat mT"].isna()
dfcores.loc[mask, "Bsat mT"] = 300
dfcores.loc[9:10,"Bsat mT"]=310 # N97=310

In [228]:
arr= []
for i in range (len(dfcores)):
    dic  =dfcores.loc[i,:].to_dict()
    Aemin = dic['Aemin(mm2)']
    K1 = dic["K1"]
    K2 = dic["K2"]
    #K3_25 = dic["K3_25"]
    #K4_25 = dic["K4_25"]
    Al_nH0 = dic["Al(nH)"]
    Le = dic["Le(mm)"]
    µe = dic["µe"]
    s_mm= (Al_nH/K1)**(1/K2) # airgap
    lg_mm= Le *(Al_nH0/Al_nH-1)/µe
    #I_dc = (0.9*Al_nH/K3_25)**(1/K4_25)
    B_peak = 1e3*Lm*sqrt(2)*Im_rms/(Np*Aemin*1e-6)
    
    µ0= 4*pi **1e-7
    N=4
    lg = lg_mm*1e-3
    le = Le*1e-3
    #Bpeak2 = µ0*N*sqrt(2)*Im_rms/(lg+le/µe)
    bsat  = dic["Bsat mT"]
    if B_peak < bsat: 
        marge= 100*(bsat-B_peak)/bsat
    else : 
        marge= np.nan
    arr.append({
        'core':dic['core'], 
        "s(mm)":s_mm,
        "lg(mm)":lg_mm,
        "B_peak": B_peak,
        #"Bpeak2":Bpeak2,
        "Bsat mT":bsat,
        "Bpeak_inf_Bsat":dic["Bsat mT"]>B_peak,
        "Bsat margin %": marge
        
    })
dfres = pd.DataFrame(arr)
dfres

,core,s(mm),lg(mm),B_peak,Bsat mT,Bpeak_inf_Bsat,Bsat margin %
0,E64/10/50-3C95,NaN,NaN,311.468182,330.0,True,5.615702
1,ELP 43/10/28 with I 43/4/28 N87,0.049951,-0.027186,744.940031,350.0,False,NaN
2,ELP 64/10/50 with I 64/5/50 N87,0.133949,0.116591,315.726537,300.0,False,NaN
3,ELP 64/10/50 with ELP 64/10/50 N95,NaN,0.118682,312.069472,320.0,True,2.478290
4,ELP 64/10/50 with ELP 64/10/50 N87,0.123167,0.110384,312.069472,300.0,False,NaN
5,E64/10/50 + PLT64/50/5 3C95,NaN,NaN,311.468182,330.0,True,5.615702
6,ER64/13/51-3C92 3C95,NaN,NaN,318.840210,330.0,True,3.381754
7,E100/60/28 N87,NaN,0.083820,234.278242,300.0,True,21.907253
8,ELP 102/20/38 with ELP 102/20/38 N87,NaN,0.082983,308.202072,300.0,False,NaN
9,ELP 102/20/38 with ELP 102/20/38 N97,NaN,0.085916,308.202072,310.0,True,0.579977


In [229]:
dfcores2 = pd.read_csv("FerriteCores.csv", sep=";").iloc[:,:-2]
df= dfcores2.merge(dfres)
mask = df.	Bpeak_inf_Bsat
df[mask].set_index("core").T

core,E64/10/50-3C95,ELP 64/10/50 with ELP 64/10/50 N95,E64/10/50 + PLT64/50/5 3C95,ER64/13/51-3C92 3C95
Aemin(mm2),519.0,518.0,519.0,507.0
Le(mm),79.9,79.9,69.7,93.0
Ve(mm3),40700,41500,35500,52600
Bsat mT,330.0,320.0,330.0,330.0
Loss (W/set),20.0,5.1,17.0,25.0
K1,NaN,NaN,NaN,NaN
K2,NaN,NaN,NaN,NaN
µe,NaN,1880.0,NaN,NaN
Al(nH),17000,15500,18500,17100
price by set,11.05,14.52,NaN,19.62


In [ ]:
# we will shose 
# ELP 64/10/50 with ELP 64/10/50 N95

## Lr design 

In [270]:
params = ['Lr_uH', 'fsw_min', 'fsw_max', 'Ir_rms']

dic_Lr= {x:out_datas[x] for x in params}
#print("Transormer parameters:")
#display(dic_Lr)

In [278]:
%%render 
Lr_uH # µH
Ir_rms # Arms

<IPython.core.display.Latex object>

In [279]:
pd.DataFrame(np.array([list(range(1,10)),
                       [1e3*Lr_uH/n**2 for n in range(1,10)]]).T,
            columns = ["Nb", "Al_nH"])

,Nb,Al_nH
0,1.0,21797.000000
1,2.0,5449.250000
2,3.0,2421.888889
3,4.0,1362.312500
4,5.0,871.880000
5,6.0,605.472222
6,7.0,444.836735
7,8.0,340.578125
8,9.0,269.098765


In [281]:
%%render long
core = "ELP 58/11/38__with __I 58/4/38 N87"
Aemin = 308 # mm2
Al_nH0 = 8400 # nH
Le= 67.7 # mm

n=4 # turns
B_peak = 1e-3*Lr_uH*sqrt(2)*Ir_rms/(n*Aemin*1e-6) # mT
B_sat = 300 # mT
B_margin = 100*(B_sat-B_peak )/B_sat  # \%
Al_nH= 1e3*Lr_uH/n**2 # nH/turn squared
K1= 591
K2= -0.685
s_mm= (Al_nH/K1)**(1/K2) # mm airgap
lg_mm= Le *(Al_nH0/Al_nH-1)/µe

<IPython.core.display.Latex object>

https://www.tdk-electronics.tdk.com/inf/80/db/fer/elp_22_6_16.pdf<br>
https://www.tdk-electronics.tdk.com/inf/80/db/fer/elp_32_6_20.pdf<br>
https://www.tdk-electronics.tdk.com/inf/80/db/fer/elp_38_8_25.pdf<br>
https://www.tdk-electronics.tdk.com/inf/80/db/fer/elp_43_10_28.pdf<br>
https://www.tdk-electronics.tdk.com/inf/80/db/fer/elp_58_11_38.pdf

In [298]:
t= """core,Ae_min(mm2)
ELP 22/6/16 with ELP 22/6/16,78.3 
ELP 32/6/20 with ELP 32/6/20,128
ELP 38/8/25 with ELP 38/8/25,192
ELP 43/10/28 with ELP 43/10/28,217
ELP 58/11/38 with ELP 58/11/38,308""".split("\n")
t= [x.split(",") for x in t]
df = pd.DataFrame(t[1:], columns = t[0])
Aemin = df["Ae_min(mm2)"].astype("float32")
B_peak = 1e-3*Lr_uH*sqrt(2)*Ir_rms/(n*Aemin*1e-6) # mT
df["B_peak(mT)"]=B_peak
df

,core,Ae_min(mm2),B_peak(mT)
0,ELP 22/6/16 with ELP 22/6/16,78.3,1019.056152
1,ELP 32/6/20 with ELP 32/6/20,128,623.375732
2,ELP 38/8/25 with ELP 38/8/25,192,415.583832
3,ELP 43/10/28 with ELP 43/10/28,217,367.705536
4,ELP 58/11/38 with ELP 58/11/38,308,259.065247


In [232]:
dfcore = pd.DataFrame([
{"core":"ELP 22/6/16 with I 22/2.5/16 N87",
 "Al_nH":5200, 
"Aemin":77.9,
 "Bsat": 300,
 "K1": 134, 
 "K2": -0.806, 
 "s_lims":[0.1,1.5]
},

{"core":"ELP 32/6/20 with I 32/3/20 N87", 
  "Al_nH": 6300, 
"Aemin":128, 
  "Bsat": 300,
 "K1":234 , 
 "K2":-0.777,
  "s_lims":[0.1,1.5]
},

{"core":"ELP 43/10/28 with I 43/4/28 N87", 
  "Al_nH": 8500, 
"Aemin":217, 
  "Bsat": 300,
 "K1":390 ,
 "K2": -0.784,
"s_lims":[0.1,2]
},


{"core":"ELP 58/11/38 with I 58/4/38 N87", 
  "Al_nH": 8400, 
"Aemin":308, 
  "Bsat": 300,
 "K1": 591 ,
 "K2":-0.685,  
   "s_lims":[0.1,1.5]
}


    


])
dfcore

,core,Al_nH,Aemin,Bsat,K1,K2,s_lims
0,ELP 22/6/16 with I 22/2.5/16 N87,5200,77.9,300,134,-0.806,"[0.1, 1.5]"
1,ELP 32/6/20 with I 32/3/20 N87,6300,128.0,300,234,-0.777,"[0.1, 1.5]"
2,ELP 43/10/28 with I 43/4/28 N87,8500,217.0,300,390,-0.784,"[0.1, 2]"
3,ELP 58/11/38 with I 58/4/38 N87,8400,308.0,300,591,-0.685,"[0.1, 1.5]"


In [233]:
dic_Lr

{'Lr_uH': 21.797, 'fsw_min': 60170.0, 'fsw_max': 156220.0, 'Ir_rms': 10.354}

In [236]:
Ir_rms = dic_Lr['Ir_rms']
Lr = dic_Lr['Lr_uH']*1e-6

arr=[]
for n in range(1,9):
    for i in range(len(dfcore)):
        Al_nH= 1e3*Lr_uH/n**2 # nH/turn squared
        Aemin = dfcore.loc[i,"Aemin"]
        K1 = dfcore.loc[i,"K1"]
        K2 = dfcore.loc[i,"K2"]
        Al_nH_core = dfcore.loc[i,"Al_nH"]
        core = dfcore.loc[i,"core"]
        s_lims= dfcore.loc[i,"s_lims"]
        Al_nH = 1000*dic_Lr["Lr_uH"]/n**2


        Al_nH_cal=Al_nH_core
        if Al_nH < Al_nH_core: 
            s_mm= (Al_nH/K1)**(1/K2)
            s_valide = s_mm>s_lims[0] and  s_mm<s_lims[1]
        else:
            s_mm= None 
            s_valide= None
        
        error = 100*(Al_nH_cal-Al_nH)/Al_nH
        
        B_peak = 1e3*Ir_rms*Lr*sqrt(2)/(n*Aemin*1e-6)
        if (s_mm is not None or abs(error)<10) and s_valide:
            arr.append({"core":core, "N":n, #"Al_nH_cal": Al_nH_cal, 
                        "Al_nH_target": Al_nH, 
                        "Al_error":error, 
                        "B_peak":B_peak, "Bsat":dfcore.loc[i, "Bsat"], "s_mm":s_mm,
                       "s_lims":s_lims, "s_valide":s_valide
                       })
dfLr=pd.DataFrame(arr).sort_values("B_peak")
dfLr

,core,N,Al_nH_target,Al_error,B_peak,Bsat,s_mm,s_lims,s_valide
10,ELP 58/11/38 with I 58/4/38 N87,6,605.472222,1287.346883,172.710172,300,0.965299,"[0.1, 1.5]",True
16,ELP 43/10/28 with I 43/4/28 N87,8,340.578125,2395.756297,183.852763,300,1.188669,"[0.1, 2]",True
6,ELP 58/11/38 with I 58/4/38 N87,5,871.880000,863.435335,207.252206,300,0.566860,"[0.1, 1.5]",True
13,ELP 43/10/28 with I 43/4/28 N87,7,444.836735,1810.813415,210.117444,300,0.845517,"[0.1, 2]",True
9,ELP 43/10/28 with I 43/4/28 N87,6,605.472222,1303.862917,245.137018,300,0.570611,"[0.1, 2]",True
3,ELP 58/11/38 with I 58/4/38 N87,4,1362.312500,516.598614,259.065257,300,0.295479,"[0.1, 1.5]",True
5,ELP 43/10/28 with I 43/4/28 N87,5,871.880000,874.904803,294.164421,300,0.358383,"[0.1, 2]",True
15,ELP 32/6/20 with I 32/3/20 N87,8,340.578125,1749.795843,311.687888,300,0.616904,"[0.1, 1.5]",True
0,ELP 58/11/38 with I 58/4/38 N87,3,2421.888889,246.836721,345.420343,300,0.127568,"[0.1, 1.5]",True
12,ELP 32/6/20 with I 32/3/20 N87,7,444.836735,1316.249943,356.214729,300,0.437468,"[0.1, 1.5]",True


## Resonanat capacitor Cr

### Chosing a capacitor for Cr

**The inputs data**

In [ ]:
%%render 
Cr_nF # nF
V_In_max # V
fsw_min # Hz

**The RMS voltage of the resonant capacitor**

In [ ]:
%%render 
X_Cr= 1/(2*pi*fsw_min*Cr_nF*1e-9) # Ohm 
V_Cr= Ir_rms*X_Cr # V
V_Cr_rms= sqrt((V_In_max/2)**2+V_Cr**2) # vrms

In [ ]:
# https://www.mouser.es/ProductDetail/EPCOS-TDK/B32672L1333J?qs=vht8qxFUkGz11LfwwLq5EA%3D%3D
# https://product.tdk.com/system/files/dam/doc/product/capacitor/film/mkp_mfp/data_sheet/20/20/db/fc_2009/mkp_b32671l_672l.pdf

Starting with the **B3267*L** film capacitors, the maximum rated DC voltage is 2000 V. [\[2\]](#referencesID_03_3).

In [ ]:
B32672L1333 = [6.2, 6.8, 8.2, 10.0, 12.0, 15.0, 22.0, 33.0, 47.0, 56.0, 68.0]
print(f"The design value of Cr is {Cr_nF:.1f} nF")
print("Below are some possible combinations for constructing this resonant capacitor:")
arr= []
for c in B32672L1333: 
    n= round(Cr_nF/c)
    ct= n*c
    error = 100*(ct-Cr_nF)/Cr_nF
    dic = {"Nominal_capa_nF":c, "Nb capas in parallel": n, 
           "total capa nF" :ct, "error %":round(error, 1)}
    arr.append(dic)
pd.DataFrame(arr)

**The current of each element capacitor**

In [ ]:
%%render 
Ir_rms_1=Ir_rms/8

**The 15 nF, 15 mm, 2000 VDC TDK MKP capacitor meets both the Vrms and Irms requirements.**

In [ ]:
first_part_path = "./03_figures/"

fig_counter = show_figure_pair_with_captions(
    fig_counter, 
    "CR_VRMS.png",
    "VRMS",
    "CR_IRMS.png",
    "IRMS", 
    first_part_path= first_part_path, 
)

Below is a screenshot from the B32672L datasheet. The ref **B32672L8153** is a 15 nF capacitor.

In [ ]:
first_part_path = "./03_figures/"

fig_counter = show_figure_pair_with_captions(
    fig_counter, 
    "capa15nF.png",
    "VRMS",
    first_part_path= first_part_path, 
)

In [ ]:
# https://eu.mouser.com/ProductDetail/EPCOS-TDK/B32672L8153J?qs=CsdphVCLJRjPN7YH24ATpg%3D%3D
# https://product.tdk.com/system/files/dam/doc/product/capacitor/film/mkp_mfp/data_sheet/20/20/db/fc_2009/mkp_b32671l_672l.pdf

In [ ]:
# B32672L8153 ==> 15n 2000VDC
# DATASHEET
# https://product.tdk.com/system/files/dam/doc/product/capacitor/film/mkp_mfp/data_sheet/20/20/db/fc_2009/mkp_b32671l_672l.pdf

**The ESR and Power dissipation**

In [ ]:
first_part_path = "./03_figures/"

fig_counter = show_figure_pair_with_captions(
    fig_counter, 
    "CR_ESR.png",
    "VRMS",
    first_part_path= first_part_path, 
)

From the below <br>
6.8nF ESR=70µΩ <br>
33nF ESR=50µΩ <br> 
By linear interpolation, the ESR of a 15 nF capacitor can be estimated as 63.74 µΩ

**The power dissipation of each capacitor**

In [ ]:
%%render 
ESR_u= np.interp([15],[6.8,33],[70,50])[0] # µΩ
Ir_rms_1 # Arms
Ir_rms_max = 2 # Arms
PD_uW = ESR_u*Ir_rms_1**2 # uW

**The Rth estimation**

In [ ]:
%%render 
T_max = 125
T_100= 100
Delta_T= T_max-T_100
PD_uW_max= ESR_u*Ir_rms_max**2
R_TH = Delta_T/(PD_uW_max) # °C/uW

**Each capacitor's self-heating and the maximum permissible ambient temperature.**

In [ ]:
%%render 
Delta_T= PD_uW*R_TH
T_amb_max = 125-Delta_T # °C 

### Understanding the Derating Curve of the Capacitors

**One point calculation**

In [ ]:
%%render 
Ta = 105
Delta_T= T_max-Ta
PD_uW_max = Delta_T/R_TH
Ir_rms_max_Ta=sqrt(PD_uW_max/ESR_u)
Fa = Ir_rms_max_Ta/Ir_rms_max

**The same calculation is repeated to construct the derating curve.**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 4))

img = plt.imread("../01_llc_tank/03_figs/Capa_thermal_derating.jpg")
axes[0].imshow(img)
axes[0].axis("off")
axes[0].set_title("Thermal Derating from datasheet")


Ta_arr= np.arange(80,125+1,1)
Fa_arr=[]
for Ta in Ta_arr:
    if Ta>100:
        Delta_T= T_max-Ta
        PD_uW_max = Delta_T/R_TH
        Ir_rms_max_Ta=sqrt(PD_uW_max/ESR_u)
        Fa = Ir_rms_max_Ta/Ir_rms_max
        Fa_arr.append(Fa)
    else: 
        Fa_arr.append(1)
        
axes[1].plot(Ta_arr, Fa_arr)

axes[1].set_xlabel("Ta (°C)")
axes[1].set_ylabel("Derating Factor (Ft)")
axes[1].set_title("Calculated Thermal Derating")

axes[1].set_xticks(np.arange(80, 125, 5))
axes[1].set_yticks(np.arange(0, 1.1, 0.1))

axes[1].grid(True)
axes[1].set_ylim([0,1.1])
axes[1].set_xlim([80,125])

### Resonant capacitors configuration

There are two possible configurations for the resonant capacitors:

- All capacitors in parallel:

which is the classic LLC configuration and offers simplicity in layout.
- Dividing the capacitors between the high side and low side:

which helps balance HV+ and HV-.

Below is a simulation of both solutions. We can see that the voltage and current of each capacitor are almost identical in both configurations, and all other voltages and currents are also very similar.

You can donwload the LTSPICE file using this <a src =".\02_simulation\03_Res_capacitor\Comparaison_Res_Capas_Archi.asc">Link</a>.

In [ ]:
first_part_path = "./03_figures/"

fig_counter = show_figure_pair_with_captions(
    fig_counter, 
    "Comparaison_Res_Capas_Archi.png",
    "VRMS",
    first_part_path= first_part_path, 
)

## References <a class="title_class" id="title_7"></a>

<a id="referencesID_03_1"></a> [1] Hong Huang, *Designing an LLC Resonant
Half-Bridge Power Converter*. Available: [https://bbs.dianyuan.com/upload/community/2013/12/01/1385867010-65563.pdf](https://bbs.dianyuan.com/upload/community/2013/12/01/1385867010-65563.pdf)

<a id="referencesID_03_2"></a> [2] 
[B40910 Aluminum electrolytic capacitors datasheet](https://product.tdk.com/system/files/dam/doc/product/capacitor/aluminum-electrolytic/hybrid-polymer/data_sheet/20/30/db/aec/b40910.pdf)

<a id="referencesID_03_3"></a> [3] 
[B3267*L Film Capacitors](https://product.tdk.com/system/files/dam/doc/product/capacitor/film/mkp_mfp/data_sheet/20/20/db/fc_2009/mkp_b32671l_672l.pdf)


<a id="referencesID_03_4"></a> [4] 
[EMHS Aluminum electrolytic capacitors datasheet](https://www.mouser.fr/datasheet/2/420/Chemi_Con_MHSRA_e-3313675.pdf)

